<a href="https://colab.research.google.com/github/Santosh-Gupta/NaturalLanguageRecommendations/blob/srihari-dev/notebooks/model_debug.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers tqdm --quiet
%tensorflow_version 2.x

You should consider upgrading via the 'pip install --upgrade pip' command.


UsageError: Line magic function `%tensorflow_version` not found.


In [1]:
import os
from tqdm.notebook import tqdm
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Lambda, Dense, Activation, Concatenate
from transformers import TFBertModel

print('TensorFlow:', tf.__version__)

TensorFlow: 2.1.0-rc2


In [2]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.MirroredStrategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
REPLICAS:  2


In [3]:
batch_size = 4 * strategy.num_replicas_in_sync
embedding_dim = 512
autotune = tf.data.experimental.AUTOTUNE

In [4]:
base_dir = '../TfRecords'
tfrecords_pattern = os.path.join(base_dir, 'train*')

In [5]:
features = {
    'title':tf.io.VarLenFeature(dtype=tf.int64),
    'citation':tf.io.FixedLenFeature([512], dtype=tf.float32),
    }

def parse_example(example_proto):
    parsed_example = tf.io.parse_single_example(example_proto, features)
    title = tf.sparse.to_dense(parsed_example['title'])
    citation = parsed_example['citation']
    
    title = tf.cast(title, dtype=tf.int32)
    citation = tf.cast(citation, dtype=tf.float32)
    title = title[:512]
    title = tf.pad(title, paddings=[[0, 512-tf.shape(title)[0]]])
    return (title, citation), tf.constant([1.0], dtype=tf.float32)

In [6]:
with strategy.scope():
    files = tf.data.Dataset.list_files(tfrecords_pattern)
    dataset = files.interleave(tf.data.TFRecordDataset,
                               cycle_length=16,
                               block_length=4,
                               num_parallel_calls=autotune)
    dataset = dataset.map(parse_example, num_parallel_calls=autotune)
    dataset = dataset.batch(batch_size, drop_remainder=True)
    dataset = dataset.prefetch(autotune)
tf.data.experimental.get_structure(dataset)

((TensorSpec(shape=(8, None), dtype=tf.int32, name=None),
  TensorSpec(shape=(8, 512), dtype=tf.float32, name=None)),
 TensorSpec(shape=(8, 1), dtype=tf.float32, name=None))

In [7]:
@tf.function
def loss_fn(_, probs):
    '''
        1. Every sample is its own positive, and  the rest of the
            elements in the batch are its negative.
        2. Each TPU core gets 1/8 * global_batch_size elements, hence
            compute shape dynamically.
        3. Dataset produces dummy labels to make sure the loss_fn matches
            the loss signature of keras, actual labels are computed inside this
            function.
        4. `probs` lie in [0, 1] and are to be treated as probabilities.
    '''
    bs = tf.shape(probs)[0] 
    labels = tf.eye(bs, bs)
    return tf.losses.categorical_crossentropy(labels, probs, from_logits=False)
    
def create_model():
    textIds = tf.keras.Input(shape=(512,), dtype=tf.int32)    # from bert tokenizer
    citation = tf.keras.Input(shape=(512,))                   # normalized word2vec outputs
    
    bert_model = TFBertModel.from_pretrained('bert-base-uncased')
    
    textOut = bert_model(textIds)
    textOutMean = tf.reduce_mean(textOut[0], axis=1)
    textOutSim = Dense(units=embedding_dim, activation='tanh', name='DenseTitle')(textOutMean)

    citationSim = Dense(units=embedding_dim, activation='tanh', name='DenseCitation')(citation)

    # Get dot product of each of title x citation combinations
    dotProduct = tf.reduce_sum(tf.multiply(textOutSim[:, None, :], citationSim), axis=-1)
    
    # Softmax to make sure each row has sum == 1.0
    probs = tf.nn.softmax(dotProduct, axis=-1)

    model = tf.keras.Model(inputs=[textIds, citation], outputs=[probs])
    return model

In [8]:
with strategy.scope():
    model = create_model()
    model.compile(loss=loss_fn, optimizer=tf.optimizers.Adam())

In [9]:
model.fit(dataset, epochs=5, steps_per_epoch=20)

Train for 20 steps
Epoch 1/5
INFO:tensorflow:batch_all_reduce: 198 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1').
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1').
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1').
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/t